In [38]:
import gc
gc.collect()
torch.cuda.empty_cache()

import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torchvision import transforms
import matplotlib.pyplot as plt
import torch
import random
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler
import pickle
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import ParameterGrid
from tqdm import tqdm
from datetime import datetime

random_seed = 1234
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)  # if use multi-GPU
#torch.backends.cudnn.deterministic = True
#torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)


In [39]:
# parameters

n_class = 63
BATCH_SIZE = 128
LR = 5e-3
Train_epoch = 1

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [40]:
X_train_ = torch.load('data/Training/X_train_diet.pt') # torch.Size([16008, 240, 320, 1])
X_test_ = torch.load('data/Testing/X_test_diet.pt') # torch.Size([1596, 240, 320, 1])

Y_train_ = pd.read_csv('data/Training/Y_train.csv')
Y_test_ = pd.read_csv('data/Testing/Y_test.csv')

# dataframe to tensor
Y_train_ = torch.tensor(Y_train_.values)  # torch.Size([16008, 63])
Y_test_ = torch.tensor(Y_test_.values)  # torch.Size([1596, 63])

In [41]:
# permute data

X_train_ = X_train_.permute([0, 3, 1, 2])
X_train_.shape

X_test_ = X_test_.permute([0, 3, 1, 2])
X_test_.shape

torch.Size([1596, 1, 240, 320])

In [42]:
def sample_and_split_dataloader(X_train_, X_test_, Y_train_, Y_test_, sample_size, valid_size, batch_size):

    # sample train set
    n_train = len(X_train_)
    indices = list(range(n_train))
    np.random.shuffle(indices)
    split = int(np.floor(sample_size * n_train))
    sample_indices = indices[:split]
    X_train_ = X_train_[sample_indices]
    Y_train_ = Y_train_[sample_indices]

    # # sample test set
    # n_test = len(X_test_)
    # indices = list(range(n_test))
    # np.random.shuffle(indices)
    # split = int(np.floor(sample_size * n_test))
    # sample_indices = indices[:split]
    # X_test_ = X_test_[sample_indices]
    # Y_test_ = Y_test_[sample_indices]

    # split train, valid set
    n_train = len(X_train_)                  
    indices = list(range(n_train))
    np.random.shuffle(indices)

    split = int(np.floor(valid_size * n_train)) 
    train_idx, valid_idx = indices[split:], indices[:split]

    train_set = TensorDataset(X_train_, Y_train_)
    train_sampler, valid_sampler = SubsetRandomSampler(train_idx), SubsetRandomSampler(valid_idx) 
    train_loader = DataLoader(train_set, batch_size=batch_size, sampler=train_sampler)
    valid_loader = DataLoader(train_set, batch_size=batch_size, sampler=valid_sampler)

    # test loader
    test_set = TensorDataset(X_test_, Y_test_)
    test_loader = DataLoader(test_set, batch_size=1)

    # print shape
    print('\n Sample size: ', sample_size)
    print('train_loader: ', len(train_loader)*train_loader.batch_size )
    print('valid_loader: ', len(valid_loader)*valid_loader.batch_size )
    print('test_loader: ', len(test_loader)*test_loader.batch_size )

    return train_loader, valid_loader, test_loader


In [43]:
train_loader, valid_loader, test_loader = sample_and_split_dataloader(
    X_train_, X_test_, Y_train_, Y_test_, sample_size=0.5, valid_size=0.2, batch_size=BATCH_SIZE)



 Sample size:  0.5
train_loader:  6528
valid_loader:  1664
test_loader:  1596


# Model

In [44]:
class my_CNN(nn.Module): 

    def __init__(self, n_class, channels=[16, 32, 64]):
        # super(my_CNN, self).__init__()
        super().__init__()
        # self.dropout - nn.Dropout(0.4)

        self.layer1 = nn.Sequential(
            nn.Conv2d(1, channels[0], kernel_size=5, stride=1, padding=2), # 240 * 320 * channels[0]
            nn.BatchNorm2d(channels[0]),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)) # 120 * 160 * channels[0]
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(channels[0], channels[1], kernel_size=5, stride=1, padding=2), # 120 * 160 * channels[1]
            nn.BatchNorm2d(channels[1]),
            nn.ReLU()
            # nn.MaxPool2d(kernel_size=2, stride=2) # 60 * 80 * channels[1]
        )

        self.layer3 = nn.Sequential(
            nn.Conv2d(channels[1], channels[2], kernel_size=5, stride=1, padding=2), # 120 * 160 * channels[2]
            nn.BatchNorm2d(channels[2]),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)  # 60 * 80 * channels[2]
        ) 
        
        self.fc = nn.Linear(60 * 80 * channels[2], n_class)

    def forward(self, x):
        # print(x.shape)  # torch.Size([1, 1, 240, 320])
        out = self.layer1(x)  
        # print(out.shape)  # torch.Size([1, 16, 120, 160])
        out = self.layer2(out)
        # print(out.shape)  # torch.Size([1, 32, 120, 160])
        out = self.layer3(out)
        # print(out.shape)  # torch.Size([1, 64, 60, 80])
        out = out.reshape(out.size(0), -1) 
        # print(out.shape) # torch.Size([1, 307200])

        out = self.fc(out) # error

        return out

In [45]:
# a = my_CNN(63)
# a(torch.randn(1, 1, 240, 320))  # B C H W 로 가짜 데이터 넣어보기


In [46]:
def train():
    
    model = my_CNN(n_class).to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr = LR)
    criterion = nn.MSELoss()
    valid_loss_min = np.inf # 초기화 (나중에 업데이트 함)

    for epoch in range(1, Train_epoch + 1): # epoch: 모든 데이터

        train_loss = 0.0
        valid_loss = 0.0

        for batch_id, (image, label) in enumerate(train_loader): # iter: batch 데이터 (25개) 
            label, image = label.to(device, dtype=torch.float), image.to(device, dtype=torch.float) # shape: (25,)
            
            output = model(image)   # 1. 모델에 데이터 입력해 출력 얻기 # 10개 클래스에 대한 로짓 # shape: (25, 10)
            loss = criterion(output, label) # 2. loss 계산 
            optimizer.zero_grad() # 3. 기울기 초기화 (iter 끝날때마다 초기화)
            loss.backward() # 4. 역전파
            optimizer.step() # 5. 최적화
        
        for batch_id, (image, label) in enumerate(valid_loader):
            label, image = label.to(device, dtype=torch.float), image.to(device, dtype=torch.float)

            output = model(image)
            loss = criterion(output, label)
            valid_loss += loss.item()
        
        # calculate avg losses
        train_loss = train_loss/len(train_loader.dataset)
        valid_loss = valid_loss/len(valid_loader.dataset)

        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch, train_loss, valid_loss))
        # save model if validation loss has decreased
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min, valid_loss))
            torch.save(model, 'best_model_CNN.pt')
            torch.save(model.state_dict(), 'best_model_CNN.pth')
            valid_loss_min = valid_loss
    
    return model

In [47]:
def test(model):

    model = torch.load('best_model_CNN.pt')  # 모델 불러오기
    print('success load best_model')
    criterion = nn.MSELoss()

    test_loss = 0.0
    with torch.no_grad():  # 파라미터 업데이트 안 함

        for batch_id, (image, label) in enumerate(test_loader):

            label, image = label.to(device, dtype=torch.float), image.to(device, dtype=torch.float)
            output = model(image)
            loss = criterion(output, label)
            test_loss += loss.item()

    # calculate avg losses
    test_loss = test_loss/len(test_loader.dataset)
    
    return test_loss

In [48]:
if __name__ == '__main__':
    
    model = train()
    test_loss = test(model)

tensor(0.3913, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(105.9126, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(5331.8037, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(1674.9882, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0409, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0384, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0387, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0402, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0402, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0379, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0392, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0381, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0403, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0396, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0384, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0399, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0403, device='cuda:0', 